In [4]:
import numpy as np
from sklearn import linear_model, datasets, metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
import pandas as pd
import os

## polyfit() function using sklearn.linear_model.RANSACRegressor ##

The polyfit function takes a csv file with equal column sizes as input<br>
Inputs:<ol>
    <li>CSV file name,</li>
    <li>polynomial order (to fit with), </li>
    <li>maxdistance (the maximum deviation from the predicted line allowed for a data point to be considered an outlier)</li></ol>
Returns:
    the x and y columns of the fitted RANSAC line


In [5]:
def polyfit(data, order, maxdistance, disable_linear = True):
    #loading and extracting columns of data for x and y
    df = pd.read_csv(data, header = None)
    dfx = df[0].values
    dfy = df[1].values
    
    #if variance of x is low, swap x and y, to prevent a vertical line
    if np.var(dfx)<np.var(dfy):
        tmp = dfx
        dfx = dfy
        dfy = tmp
    
    x = np.reshape(dfx, (len(dfx),1))
    
    y = np.reshape(dfy,(len(dfy),))
    
    #creation of the RANSACRegressor object
    ransac = make_pipeline(PolynomialFeatures(order), linear_model.RANSACRegressor(residual_threshold = maxdistance))
    
    ransac.fit(x,y)
    
    #creation of boolean mask arrays to indicate the (x,y) pairs that are inliers vs. outliers
    
    
    line_x = np.linspace(x.min(), x.max(), len(x))[:, np.newaxis]

    line_y_ransac = ransac.predict(line_x)
    
    #additional linear fit
    linear = linear_model.RANSACRegressor()
    linear.fit(x,y)
    
    inlier_mask = linear.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    
    line_y_linear = linear.predict(line_x)

    lw = 2

    plt.scatter(x,y, color="red", marker = '.')

    
    #select best fitted line using mean squared error
    
    
    MSE_regressor = metrics.mean_squared_error(dfy[:len(dfy)], line_y_ransac)
    MSE_linear = metrics.mean_squared_error(dfy[:len(dfy)], line_y_linear)
    
    print("MSE of the linear fit: "+str(MSE_linear))
    print("MSE of the polynomial fit: "+str(MSE_regressor))
    
    #determine the model to use based on lowest MSE
    
    chosen_model = None
    alternative_model = None
    if disable_linear == False:
    
        if (MSE_linear < MSE_regressor):
            chosen_model = linear
            alternative_model = ransac
            print("Linear model chosen")
            
            plot_line(line_x, line_y_linear, 'LinearRegressor',lw, color = "cornflowerblue")
            
        else:
            chosen_model = ransac
            alternative_model = linear
            print("Polynomial model chosen")
            
            plot_line(line_x, line_y_ransac, 'RANSACRegressor',lw, color = "cornflowerblue")

    else:
        chosen_model = ransac
        alternative_model = linear
        print("Polynomial model chosen")
        
        plot_line(line_x, line_y_ransac, 'RANSACRegressor',lw, color = "cornflowerblue")
    
    #information to be returned x_coords, y_coords(linear prediction), y_coords(polynomial prediction), chosen sklearn model, unchosen sklearn model
    cache = {"x":line_x, "yl":line_y_linear, "yr": line_y_ransac, "model": chosen_model, "alt_model": alternative_model}
    
    return cache

In [6]:
def plot_line(x,y, label, lw ,color = 'blue'):
    plt.plot(x, y, color=color, linewidth=lw,
    label=label)
    plt.legend(loc='lower right')
    plt.xlabel("Input")
    plt.ylabel("Response")
    
    xmin,xmax = plt.xlim()
    ymin,ymax = plt.ylim()
    
    if (xmax-xmin > ymax-ymin):
        plt.ylim((ymax+ymin)/2-(xmax-xmin)/2,(ymax+ymin)/2+(xmax-xmin)/2)
    else:
        plt.xlim((xmax+xmin)/2-(ymax-ymin)/2,(xmax+xmin)/2+(ymax-ymin)/2)
    print(plt.xlim())
    print(plt.ylim())
    
    plt.axis("equal")

    

## arclength() <br>

Takes input:
<ol>
    output cache of polyfit()
    </ol><br>
Returns:<br>
<ol>
    a cache containing the custom x coordinates, the y output of arclength as a function of each x coordinate, and the arclength

In [7]:
def arclength(cache, linespace = 5000):
    line_x = cache["x"]
    x_coords = np.linspace(line_x.min(),line_x.max(),linespace)[:, np.newaxis]
    y_coords = cache["model"].predict(x_coords)
    
    s_accumulative = 0.0
    s_list = []
    
    
    for i in range(len(x_coords)-1):
        x_tmp = x_coords[i]
        y_tmp = y_coords[i]
        
        x_next = x_coords[i+1]
        y_next = y_coords[i+1]
    
        
        s_accumulative += np.sqrt(np.power((x_next-x_tmp),2)+np.power((y_next-y_tmp),2)).item()
        s_list.append(s_accumulative)
    
    return {"x": x_coords, "s(x)": s_list, "arclength":s_accumulative}

## spacing() ##
Inputs: <br>
 <ol> output from arclength(), the arclength between adjacent units (such as a particle in a microtubule) </ol><br>
Outputs: <br>
 <ol> the respective x coordinates of the curve such that between each x coordinate, the change in arclength is constant. </ol>

In [8]:
def spacing(cache, step_size):
    #based on the number of segments you want to divide the arc into, return the respective x coordinate at each segment
    
    l = np.arange(0,cache["arclength"], float(step_size))[:, np.newaxis]
    
    s = np.asarray(cache["s(x)"])
    
    x_locations = []
    for val in l:
        idx = np.abs(s-val).argmin()
        x_locations.append(cache["x"][idx])
        
    return x_locations
    

## process_data() ##
This is the only function you need to call to perform interpolation + evenly spaced particle picking along the interpolated line<br>
Inputs:<br>
<ol>
    <li>directory - directory name(string) that contains csv files of the picked coordiantes</li>
    <li>pixel_dist - distance along the arclength between adjacent units(this would be the spacing between two particles, for example)</li>
    <li>disable_linear - set to True if only doing RANSAC regression, False if doing both linear and RANSAC regression and choosing the one with better fit</li>
    <li>indexi - by default 0, the start position of the csv file you want to process</li>
    <li>indexj - by default 10, the position of the last csv file you want to process</li>
    <li>parse_whole_dataset - by default False, if set to True, indexi and indexj will be automatically set to 0 and the length of all the files in the directory as a list </li>
</ol>
<br>
Outputs:<br>
    <li>Does not return any value, instead, stores the interpolation graph and the coordiantes of adjacent units into two directories called test_data_plots and test_data_positions, respectively.(Make sure these directories are created in the current directory)</li>
    

In [14]:
def process_data(directory, pixel_dist ,disable_linear = True, indexi = 0, indexj = 10, parse_whole_dataset = False):
    file_list = os.listdir(directory)
    subset = [x for x in file_list if ".txt" in x]
    if (parse_whole_dataset == True):
        indexi = 0
        indexj = len(subset)
    
    for i in range(indexi,indexj):
        poly_o = polyfit(directory+"/"+subset[i],2,1, disable_linear = disable_linear)
        arclength_o = arclength(poly_o)
        output = spacing(arclength_o, pixel_dist)
        
        plt.scatter(output, poly_o["model"].predict(output))
        plt.savefig("test_data_plots/"+subset[i]+"_plot.png")    
        plt.clf()
        
        unit_positions = pd.DataFrame({'x':output, 'y': poly_o["model"].predict(output)})
        unit_positions.to_csv(r"test_data_positions/"+subset[i]+"_unit_pos.txt", header = False, index = False)

    

Example:

In [13]:
print("data.txt:")
print(pd.read_csv("data.txt", index_col =  None, header = None).to_string(header = ["x","y"], index = False))

cache = polyfit("data.txt",2,0.5,disable_linear = False)

al = arclength(cache)
print("arclength:"+str(al["arclength"]))
#x_coordinates of evenly spaced segments
output = spacing(al, 60.218)
print(output)

print(cache["model"].predict(output))

plt.scatter(output, cache["model"].predict(output))

pd.DataFrame({'x':output, 'y': cache["model"].predict(output)})

#plt.plot(x,y)

#pd.DataFrame(x,y)

data.txt:


FileNotFoundError: [Errno 2] File b'data.txt' does not exist: b'data.txt'

In [16]:
process_data("../test_data/test_data", 60.218, disable_linear = False)

MSE of the linear fit: 462.62261566604167
MSE of the polynomial fit: 462.6094482747513
Polynomial model chosen
(1433.6017953345877, 3842.3026036654123)
(1081.5602658345879, 3490.2610741654125)
MSE of the linear fit: 130609.52634073097
MSE of the polynomial fit: 130596.83720497937
Polynomial model chosen
(238.82980353458782, 2249.7424034654123)
(3189.406785395921, 5200.319385326746)
MSE of the linear fit: 346.09366164714504
MSE of the polynomial fit: 348.68875653395855
Linear model chosen
(393.93255003458773, 1925.741766965412)
(2732.220087651888, 4264.029304582712)
MSE of the linear fit: 940.8097538217254
MSE of the polynomial fit: 1009.3855516686568
Linear model chosen
(3679.6117817845875, 5074.288998215414)
(1033.0218191105357, 2427.699035541362)
MSE of the linear fit: 39160.36419921823
MSE of the polynomial fit: 38106.330917857966
Polynomial model chosen
(3710.721330084588, 4574.824539915413)
(27.48705808458749, 891.5902679154126)
MSE of the linear fit: 64.60854446984271
MSE of the 

<Figure size 432x288 with 0 Axes>

tip: if the variation of x is less than y axis, flip the axis.